In [1]:
import torch
import torch.nn as nn
import numpy as np
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os    
import mediapipe as mp
import torch.nn.functional as F
import cv2
import time
import webbrowser
import keyboard
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
class Dataset(Dataset):
    def __init__(self, *args): #получаем n массивов данных
        cnt, labels, temp =0, [], []
        self.data = []
        for data in args: #выделяем по одному из них
            self.data.extend(torch.tensor(data).type(torch.float32)) # преобразуем каждый в тензор и добавляем к общим данным
            print(len(data))
            for j in range(len(data)):
                labels.append(cnt) #присовим номера каждому подмассиву для проверки
            cnt += 1
        self.labels = labels
            
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.data)

In [3]:
data, temp = [],[]
gests = os.listdir('dataset3')
for gest in gests:
    data = []
    print(gest)
    f = open('dataset3/' + str(gest)).readline()
    f = json.loads(f)
    for i in range (len(f)):
        data.append(f[i])
    temp.append(data)
data = np.array(temp,dtype=object)

r_ladon_d.txt
r_kulak_d.txt
l_ladon_d2.txt
thrashhhh3.txt
l_kulak_d.txt


In [4]:
dataset = Dataset(*data)
dataloader = DataLoader(dataset, shuffle=True, batch_size = 16)

2001
2001
2001
4502
2001


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(42, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 5)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x),negative_slope=0.001)
        x = F.leaky_relu(self.fc2(x),negative_slope=0.001)
        x = self.fc4(x)
        return x

In [6]:
net = Net()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5)

In [8]:
for epoch in range(30):
    running_loss = 0.0
    for samples, labels in dataloader:
        pred = net(samples)
        loss = criterion(pred, labels)
        
        loss.backward()
        optimizer.step()
        
        optimizer.zero_grad()
        running_loss += loss.item()
    print(running_loss/len(dataloader))
print('Finished Training')

0.6248380338768368
0.3214070470646367
0.2611926216775042
0.21537422924719352
0.19643641737363565
0.18350598846843272
0.15423968634115595
0.1362630572006859
0.13982908712382025
0.12419577117216156
0.11599773443524328
0.106189834473975
0.11707442174987186
0.10024729044229636
0.0932653091395038
0.09343302126272994
0.08857117407031351
0.08512090254322532
0.09003767335292874
0.08849741425945798
0.082155254713106
0.08463158383501022
0.07253063660029553
0.07683192874284704
0.07284515746750052
0.06963834617606393
0.07277996040774388
0.06511105002580903
0.06579285179754026
0.06968648693856799
Finished Training


In [9]:
ls=0
for samples, labels in dataloader:
    pred = net(samples)
    v,i = torch.max(pred, 1)
    ls+=torch.sum(abs(labels - i)).item()

print((len(dataset)-ls)/len(dataset))

torch.save(net,"1.pth" )

0.9708939708939709


In [10]:
pred = net(samples)
v,i = torch.max(pred, 1)

In [11]:
hands = mp.solutions.hands.Hands(static_image_mode=False,
                         max_num_hands=2,
                         min_tracking_confidence=0.5,
                         min_detection_confidence=0.5)

mp_pose = mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

def img_text(img, text, pos):
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [12]:
#watch video
names = ['r_ladon', 'r_kulak', 'l_ladon','thrash','l_kulak']
cap = cv2.VideoCapture(0)
cnt_iter=[]
check=True
check_arr=[]
set_play_pos=True
while(True):
    _, img = cap.read()
    h, w, _ = img.shape
    img = cv2.flip(img, 1)
    # hands
    t = time.time()
    result = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    hand_time = time.time() - t
   
    if result.multi_hand_landmarks:
        for_model = []
        for id, lm in enumerate(result.multi_hand_landmarks[0].landmark):
            cx, cy = int(lm.x*w), int(lm.y*h)
            cv2.circle(img, (cx, cy), 5, (0, 255, 0))
            for_model.append(lm.x)
            for_model.append(lm.y)
        for_model = torch.tensor([for_model]).type(torch.float)
        pred = net(for_model)
        v,i = torch.max(pred, 1)
        img_text(img, str(names[i.item()]), (100, 30))
        cnt_iter.append(i.item())
        check_arr.append(i.item())
        if len(cnt_iter)==5:
            vals, counts = np.unique(cnt_iter, return_counts=True)
            #print(vals[np.argmax(counts)])
            cnt_iter=[]
            if check==True:
                match int(vals[np.argmax(counts)]):
                    case 0:
                        webbrowser.open("https://www.youtube.com/watch?v=dQw4w9WgXcQ&ab_channel=RickAstley")
                        print('r_ladon')
                        check=False
                    case 1:
                        webbrowser.open("https://www.youtube.com/watch?v=A1Qb4zfurA8&ab_channel=Enjoykipip")
                        print('r_kulak')
                        check=False
                    case 2:
                        print('l_ladon')
                        if set_play_pos==True:
                            keyboard.send("space")
                            set_play_pos=False
                        check=False
                    case 3:
                        print('thrash')
                    case 4:
                        print('l_kulak')
                        if set_play_pos==False:
                            keyboard.send("space");
                            set_play_pos=True
                        check=False
            else:
                if len(check_arr)>=10:
                    check=True
                    check_arr=[]
        

    cv2.imshow("1",img)
    if cv2.waitKey(1)>-1:
        break
     
cv2.destroyAllWindows()

r_ladon
l_ladon
l_ladon
l_kulak
l_kulak
l_ladon
r_kulak
r_kulak
l_ladon
l_ladon
l_ladon
l_ladon
r_ladon
l_kulak
l_ladon
l_ladon
l_kulak
